# Patient Data Analysis

This notebook demonstrates how to connect to the PostgreSQL database and analyze patient data using Python and SQL.

## Setup

First, let's import the necessary libraries and set up the database connection.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta

# Install matplotlib
%pip install matplotlib

import matplotlib.pyplot as plt

# Set up plotting
%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Import our database configuration
import sys
sys.path.append('../connection')
from db_config import query_to_dataframe, get_connection

## Patient Demographics

Let's analyze the age distribution of our patients.

In [ ]:
# Query to get patient age distribution
age_query = """
SELECT 
    EXTRACT(YEAR FROM AGE(NOW(), u.\"dateOfBirth\")) AS age,
    COUNT(*) AS patient_count
FROM 
    users u
JOIN 
    patient_profiles pp ON u.id = pp.\"userId\"
WHERE 
    u.role = 'PATIENT'
GROUP BY 
    age
ORDER BY 
    age;
"""

# Execute the query and load results into a DataFrame
age_df = query_to_dataframe(age_query)

# Display the first few rows
age_df.head()

In [ ]:
# Plot the age distribution
plt.figure(figsize=(12, 6))
sns.barplot(x='age', y='patient_count', data=age_df)
plt.title('Patient Age Distribution')
plt.xlabel('Age')
plt.ylabel('Number of Patients')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Appointment Analysis

Let's analyze appointment data to understand patterns and trends.

In [ ]:
# Query to get appointment statistics by month
appointment_query = """
SELECT 
    DATE_TRUNC('month', \"appointmentTime\") AS month,
    \"appointmentType\",
    COUNT(*) AS appointment_count,
    COUNT(CASE WHEN status = 'COMPLETED' THEN 1 END) AS completed,
    COUNT(CASE WHEN status = 'CANCELLED' THEN 1 END) AS cancelled,
    COUNT(CASE WHEN status = 'NO_SHOW' THEN 1 END) AS no_show
FROM 
    appointments
GROUP BY 
    month, \"appointmentType\"
ORDER BY 
    month DESC, \"appointmentType\";
"""

# Execute the query and load results into a DataFrame
appointment_df = query_to_dataframe(appointment_query)

# Convert month to datetime and format it
appointment_df['month'] = pd.to_datetime(appointment_df['month'])
appointment_df['month_formatted'] = appointment_df['month'].dt.strftime('%b %Y')

# Display the first few rows
appointment_df.head()

In [ ]:
# Plot appointment trends over time
plt.figure(figsize=(14, 7))

# Pivot the data for plotting
appointment_pivot = appointment_df.pivot_table(
    index='month', 
    columns='appointmentType', 
    values='appointment_count',
    aggfunc='sum'
)

# Plot
appointment_pivot.plot(kind='line', marker='o')
plt.title('Appointment Trends by Type')
plt.xlabel('Month')
plt.ylabel('Number of Appointments')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Medication Analysis

Let's analyze medication prescription patterns.

In [ ]:
# Query to get medication prescription trends
medication_query = """
SELECT 
    \"medicationName\",
    COUNT(*) AS prescription_count,
    COUNT(DISTINCT \"patientId\") AS patient_count
FROM 
    medications
WHERE 
    status = 'ACTIVE'
GROUP BY 
    \"medicationName\"
ORDER BY 
    prescription_count DESC
LIMIT 20;
"""

# Execute the query and load results into a DataFrame
medication_df = query_to_dataframe(medication_query)

# Display the first few rows
medication_df.head()

In [ ]:
# Plot top medications
plt.figure(figsize=(12, 8))
sns.barplot(x='prescription_count', y='medicationName', data=medication_df.head(10))
plt.title('Top 10 Most Prescribed Medications')
plt.xlabel('Number of Prescriptions')
plt.ylabel('Medication')
plt.tight_layout()
plt.show()

## Provider Workload Analysis

Let's analyze provider workload to understand resource allocation.

In [ ]:
# Query to get provider workload
provider_query = """
SELECT 
    p.id AS provider_id,
    u.\"firstName\" || ' ' || u.\"lastName\" AS provider_name,
    p.specialty,
    COUNT(a.id) AS total_appointments,
    COUNT(DISTINCT a.\"patientId\") AS unique_patients
FROM 
    providers p
JOIN 
    users u ON p.\"userId\" = u.id
LEFT JOIN 
    appointments a ON p.id = a.\"providerId\"
WHERE 
    a.\"appointmentTime\" BETWEEN CURRENT_DATE - INTERVAL '30 days' AND CURRENT_DATE
GROUP BY 
    p.id, provider_name, p.specialty
ORDER BY 
    total_appointments DESC;
"""

# Execute the query and load results into a DataFrame
provider_df = query_to_dataframe(provider_query)

# Display the first few rows
provider_df.head()

In [ ]:
# Plot provider workload
plt.figure(figsize=(14, 8))

# Create a scatter plot
sns.scatterplot(
    x='total_appointments', 
    y='unique_patients', 
    hue='specialty', 
    size='total_appointments',
    sizes=(50, 500),
    alpha=0.7,
    data=provider_df
)

# Add provider names as annotations
for i, row in provider_df.iterrows():
    plt.annotate(
        row['provider_name'], 
        (row['total_appointments'], row['unique_patients']),
        xytext=(5, 5),
        textcoords='offset points'
    )

plt.title('Provider Workload Analysis (Last 30 Days)')
plt.xlabel('Total Appointments')
plt.ylabel('Unique Patients')
plt.grid(True)
plt.tight_layout()
plt.show()

## Custom Analysis

You can add your own custom analysis below. Here's a template for running a custom SQL query:

In [ ]:
# Define your custom SQL query
custom_query = """
-- Your SQL query here
SELECT * FROM your_table LIMIT 10;
"""

# Execute the query and load results into a DataFrame
# custom_df = query_to_dataframe(custom_query)

# Display the results
# custom_df.head()

## Conclusion

This notebook demonstrates how to connect to the PostgreSQL database and analyze patient data using Python and SQL. You can extend this analysis by adding more queries and visualizations.